In [97]:
import pandas as pd

coef_df = pd.read_csv('coef.csv')
print(coef_df.isna().sum())
coef_df = coef_df.fillna('NA')
print(coef_df.isna().sum())
coef_df

y                  0
x                  0
mean               0
std                0
p-value            0
FDR adj p-value    0
dtype: int64
y                  0
x                  0
mean               0
std                0
p-value            0
FDR adj p-value    0
dtype: int64


,y,x,mean,std,p-value,FDR adj p-value
0,Female,Female,0.993232,0.000163,0.000000,0.000000
1,Deceased,Age 0-4,-0.028898,0.002594,0.000010,0.001717
2,Deceased,Age 10-14,-0.028633,0.004566,0.000416,0.042578
3,Deceased,Age 20-24,-0.022250,0.002961,0.000136,0.016237
4,Deceased,Age 65-69,0.035180,0.004351,0.000085,0.010849
...,...,...,...,...,...,...
149,Alzheimer's disease (disorder),Age 80-84,0.015801,0.002467,0.000365,0.038954
150,Hypertriglyceridemia (disorder),Diabetes,0.303065,0.020496,0.000002,0.000337
151,Metabolic syndrome X (disorder),Diabetes,0.212059,0.019241,0.000011,0.001816
152,Recurrent rectal polyp,Polyp of colon,0.078191,0.010453,0.000140,0.016582


In [98]:
constant_df = pd.read_csv('constant.csv')
constant_df['label'] ='constant'
print(constant_df.isna().sum())
constant_df = constant_df.fillna('NA')
print(constant_df.isna().sum())

constant_df

name               0
mean               0
std                0
p-value            0
FDR adj p-value    0
label              0
dtype: int64
name               0
mean               0
std                0
p-value            0
FDR adj p-value    0
label              0
dtype: int64


,name,mean,std,p-value,FDR adj p-value,label
0,Female,0.003271,0.000172,2.721592e-07,0.058590,constant
1,Deceased,0.040106,0.003621,1.087478e-05,0.000018,constant
2,Age 0-4,0.000644,0.001340,6.455327e-01,0.166165,constant
3,Age 5-9,0.000542,0.000888,5.611949e-01,0.001048,constant
4,Age 10-14,0.001015,0.001737,5.774694e-01,0.008502,constant
...,...,...,...,...,...,...
129,"Localized, primary osteoarthritis of the hand",-0.000835,0.003062,7.928603e-01,0.999171,constant
130,Osteoarthritis of hip,0.000176,0.000603,7.785546e-01,0.999171,constant
131,Contact dermatitis,0.000372,0.000798,6.550815e-01,0.926072,constant
132,Concussion injury of brain,0.002081,0.005078,6.942439e-01,0.935836,constant


In [47]:
#server = 'http://67.205.191.70'
server = 'https://yaniv.i.tgcloud.io'

import getpass

password = getpass.getpass()

········


In [9]:

!pip install pyTigerGraph

In [48]:
import pyTigerGraph as tg 

# Replace graphname='social' with the graph you would like to be used as default
conn = tg.TigerGraphConnection(
    host=server, 
    graphname='ML', 
    password=password)

# Change client_version once TigerGraph Cloud is upgraded  
#shell = tg.Gsql(conn, certNeeded=False)
shell = tg.Gsql(conn, certNeeded=True)

In [111]:
print(shell.gsql('ls', options=[]))

Trying version: v3_0_0
Connecting to yaniv.i.tgcloud.io:14240
GSQL client is not compatible with GSQL server. Please contact support@tigergraph.com
Supported Versions ( v3_0_0 v2_6_0 v2_5_2 v2_5_0 v2_4_1 v2_4_0 v2_3_2 )
You may use 'GSQL_CLIENT_VERSION=v? java ...' or 
    'java -DGSQL_CLIENT_VERSION=v? ...' to specify the version
Trying version: v2_6_0
Connecting to yaniv.i.tgcloud.io:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
---- Global vertices, edges, and all graphs
Vertex Types: 
  - VERTEX Patient(PRIMARY_ID patient_id STRING, name STRING, gender STRING, birth DATETIME, death DATETIME) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  - VERTEX Provider(PRIMARY_ID provider_id STRING, name STRING, specialty STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  - VERTEX Encounter(PRIMARY_ID encounter_id STRING, description STRING, startTime DATETIME, endTime DATETIME) WITH STATS="OUTDEGREE_BY_EDGETYPE", P

In [55]:
secret = shell.createSecret()
len(secret)

32

In [57]:
token = conn.getToken(secret=secret)
len(token[0])

32

In [112]:
print(shell.gsql('''
    drop graph ML
    drop edge ML_reg_constant
    drop edge ML_reg_coef
    drop vertex ML_condition
    drop vertex ML_label
''', options=[]))

Trying version: v3_0_0
Connecting to yaniv.i.tgcloud.io:14240
GSQL client is not compatible with GSQL server. Please contact support@tigergraph.com
Supported Versions ( v3_0_0 v2_6_0 v2_5_2 v2_5_0 v2_4_1 v2_4_0 v2_3_2 )
You may use 'GSQL_CLIENT_VERSION=v? java ...' or 
    'java -DGSQL_CLIENT_VERSION=v? ...' to specify the version
Trying version: v2_6_0
Connecting to yaniv.i.tgcloud.io:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The graph ML is dropped.
The edge type ML_reg_constant is dropped.
The edge type ML_reg_coef is dropped.
The vertex type ML_condition is dropped.
The vertex type ML_label is dropped.



In [113]:
print(shell.gsql('''
    create vertex ML_condition(primary_id condition_id string) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
    create vertex ML_label(primary_id label_id string) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
    create undirected edge ML_reg_constant(from ML_condition, to ML_label, name string, mean double, std double, p_value double, adj_p_value double)   
    create directed edge ML_reg_coef(from ML_condition, to ML_condition, x string, y string, mean double, std double, p_value double, adj_p_value double) 
    
    create graph ML (ML_condition, ML_label, ML_reg_constant, ML_reg_coef)
''', options=[]))

Trying version: v3_0_0
Connecting to yaniv.i.tgcloud.io:14240
GSQL client is not compatible with GSQL server. Please contact support@tigergraph.com
Supported Versions ( v3_0_0 v2_6_0 v2_5_2 v2_5_0 v2_4_1 v2_4_0 v2_3_2 )
You may use 'GSQL_CLIENT_VERSION=v? java ...' or 
    'java -DGSQL_CLIENT_VERSION=v? ...' to specify the version
Trying version: v2_6_0
Connecting to yaniv.i.tgcloud.io:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The vertex type ML_condition is created.
The vertex type ML_label is created.
The edge type ML_reg_constant is created.
The edge type ML_reg_coef is created.
The graph ML is created.



In [114]:
conn.upsertVertex(vertexType='ML_label', vertexId='constant', attributes={'label_id': 'constant'})

1

In [115]:
conn.upsertVertexDataframe(df=constant_df, vertexType='ML_condition', v_id='name', attributes={'condition_id':'name'})

134

In [116]:
conn.upsertEdgesDataframe(
    df=constant_df, 
    sourceVertexType='ML_condition', 
    edgeType='ML_reg_constant', 
    targetVertexType='ML_label', 
    from_id='name', 
    to_id='label', 
    attributes={
        'name':'name', 'mean':'mean', 'std':'std', 
        'p_value':'p-value', 'adj_p_value': 'FDR adj p-value'},
)


134

In [117]:
conn.upsertEdgesDataframe(
    df=coef_df, 
    sourceVertexType='ML_condition', 
    edgeType='ML_reg_coef', 
    targetVertexType='ML_condition', 
    from_id='x', 
    to_id='y', 
    attributes={
        'x':'x', 'y':'y', 'mean':'mean', 'std':'std', 
        'p_value':'p-value', 'adj_p_value': 'FDR adj p-value'},
)

154